In [1]:
import sqlite3
import pandas
import data_algebra.cdata
import data_algebra.SQLite
from data_algebra.data_ops import *
from data_algebra.data_pipe import *

In [2]:
iris = pandas.read_csv('iris.csv')
iris = iris.iloc[range(3), :]
iris.reset_index(inplace=True, drop=True)
iris

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa


In [3]:
# from:
#   https://github.com/WinVector/cdata/blob/master/vignettes/control_table_keys.Rmd

control_table = pandas.DataFrame({
    'Part': ["Sepal", "Sepal", "Petal", "Petal"],
    'Measure': ["Length", "Width", "Length", "Width"],
    'Value': ["Sepal.Length", "Sepal.Width", "Petal.Length", "Petal.Width"]
})
control_table

,Part,Measure,Value
0,Sepal,Length,Sepal.Length
1,Sepal,Width,Sepal.Width
2,Petal,Length,Petal.Length
3,Petal,Width,Petal.Width


In [4]:
record_spec = data_algebra.cdata.RecordSpecification(
    control_table,
    control_table_keys = ['Part', 'Measure'],
    record_keys = ['Species']
    )
record_spec

RecordSpecification
   record_keys: ['Species']
   control_table_keys: ['Part', 'Measure']
   control_table:
       Part Measure         Value
   0  Sepal  Length  Sepal.Length
   1  Sepal   Width   Sepal.Width
   2  Petal  Length  Petal.Length
   3  Petal   Width   Petal.Width

In [5]:
db_model = data_algebra.SQLite.SQLiteModel()

source_table = data_algebra.data_ops.TableDescription(
    'iris',
    ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width', 'Species']
    )

temp_table = data_algebra.data_ops.TableDescription(
    'control_table',
    record_spec.control_table.columns
    )
    
conn = sqlite3.connect(':memory:')
cur = conn.cursor()


sql = db_model.row_recs_to_blocks_query(source_table, record_spec, temp_table)
print(sql)

db_model.insert_table(conn, iris, 'iris')
db_model.insert_table(conn, record_spec.control_table, temp_table.table_name)

db_model.read_query(conn, sql)

SELECT
 a."Species" AS "Species",
 b."Part" AS "Part",
 b."Measure" AS "Measure",
 CASE
  WHEN b."Value" = 'Sepal.Length' THEN a."Sepal.Length"
  WHEN b."Value" = 'Sepal.Width' THEN a."Sepal.Width"
  WHEN b."Value" = 'Petal.Length' THEN a."Petal.Length"
  WHEN b."Value" = 'Petal.Width' THEN a."Petal.Width"
 ELSE NULL END AS "Value"
 FROM "iris" a CROSS JOIN "control_table" b



,Species,Part,Measure,Value
0,setosa,Sepal,Length,5.1
1,setosa,Sepal,Width,3.5
2,setosa,Petal,Length,1.4
3,setosa,Petal,Width,0.2
4,setosa,Sepal,Length,4.9
5,setosa,Sepal,Width,3.0
6,setosa,Petal,Length,1.4
7,setosa,Petal,Width,0.2
8,setosa,Sepal,Length,4.7
9,setosa,Sepal,Width,3.2


In [6]:
conn.close()